In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

In [11]:
df = pd.read_excel('var_apl.xlsx', sheet_name=2)
df = df.set_index('Date')
df = df.sort_index()
df.head()

,Bid,Ask,Bid.1,Ask.1
Date,,,,
2022-03-04,114.6900,115.0600,4.9350,4.9400
2022-03-07,120.2600,120.5000,4.8310,4.8540
2022-03-08,124.2300,124.8300,4.6170,4.6500
2022-03-09,109.2000,110.5900,4.5350,4.6000
2022-03-10,105.5700,105.8500,4.6500,4.7310


In [ ]:
conf = 99
long = False
posiciones = np.array([1000*1245, 10000*871])

In [9]:
def var_apl(df, posiciones, conf, long):

    # Bid y Ask
    bid_columns = [col for col in df.columns if 'Bid' in col]
    ask_columns = [col for col in df.columns if 'Ask' in col]

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))]
    df[mid_columns] = (df[bid_columns].values + df[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))]
    df[spread_columns] = (df[ask_columns].values - df[bid_columns].values) / df[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))]
    df[return_columns] = df[mid_columns].pct_change()

    # Weights
    value = posiciones * df[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    df['port_ret'] = np.dot(df[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(df['port_ret'].dropna(), 100 - conf) if long else np.percentile(df['port_ret'].dropna(), conf)
    var_cash = pv * var_pct

    # Liquidity cost
    cl_prom = df[spread_columns].mean()
    cl_estr = np.percentile(df[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = ((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr)))

    var_apl_prom_cash, var_apl_estr_cash = ((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr)))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado'],
        'Porcentaje': [var_pct, var_apl_prom, var_apl_estr],
        'Cash': [var_cash, var_apl_prom_cash, var_apl_estr_cash]
    })

    return resultados

In [10]:
var_apl(df, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0161,"351,914,316.5583"
1,VaR Ajustado Promedio,0.0163,"357,331,030.9419"
2,VaR Ajustado Estresado,0.0166,"363,666,029.1972"
